## Overview ##

1. 19 columns were identified as having NULL values, out of which 4 columns have more than 70 percent were NULL, which were dropped
2. The following attributes were chosen based on our initial correlation analysis; which are some of the common attributest between test datasets 
   LotArea , BldgType,  HouseStyle ,  YearBuilt,    FullBath,   HalfBath ,   BedroomAbvGr
3. Another correlation analysis was performed among the selected attributes to avoid multicollinearity 
4. In order to avoid outliers, LotArea greater than 50,000 sq.ft. were eliminated 
5. Test datasets are gathered from Delaware- Bear, Delaware- Newark, Delaware-Wilmington and the latest data from Iowa- Ames.
6. Full Bath, Half Bath, Year Built and lot area are the most significant predictors in the model
7. With this prediction model, predicted house price  is off by an average of  $53,600
8. The R^2 statistic shows how well the model explains SalePrice.
9. In this model, since R^2 and Adjusted R^2 are close, model is not overfit.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

import seaborn as sns
from sklearn import metrics

import cpi
cpi.update()

### Data Set

In [ ]:
housingData=pd.read_csv('Data/housing.csv')
housingData.head()

## Basic EDA

In [ ]:
housingData.shape

In [ ]:
housingData.info()

### Attribute Correlation Metrics

In [ ]:
corr = housingData.corr()
corr.style.background_gradient()

### FInd missing Data

In [ ]:
def find_missing_percent(data):
    """
    Returns dataframe containing the total missing values and percentage of total
    missing values of a column.
    """
    miss_df = pd.DataFrame({'ColumnName':[],'TotalMissingVals':[],'PercentMissing':[]})
    for col in data.columns:
        sum_miss_val = data[col].isnull().sum()
        percent_miss_val = round((sum_miss_val/data.shape[0])*100,2)
        miss_df = miss_df.append(dict(zip(miss_df.columns,[col,sum_miss_val,percent_miss_val])),ignore_index=True)
    return miss_df

In [ ]:
miss_df = find_missing_percent(housingData)
'''Displays columns with missing values'''
display(miss_df[miss_df['PercentMissing']>0.0])
print("\n")

print("Number of columns with missing values:"+(str(miss_df[miss_df['PercentMissing']>0.0].shape[0])))

In [ ]:
drop_cols = miss_df[miss_df['PercentMissing'] >70.0].ColumnName.tolist()

print("Number of columns with more than 70%:"+ str(len(drop_cols)))
housingData = housingData.drop(drop_cols,axis=1)
#test = test.drop(drop_cols,axis =1)

miss_df = miss_df[miss_df['ColumnName'].isin(housingData.columns)]
'''Columns to Impute'''
impute_cols = miss_df[miss_df['TotalMissingVals']>0.0].ColumnName.tolist()
miss_df[miss_df['TotalMissingVals']>0.0]

### Basic Stats

In [ ]:
housingData.describe().transpose()

In [ ]:
freq1, bin_edges1=np.histogram(housingData.LotArea, bins='fd')

plt.figure(figsize=(15,5))  
ax1 = plt.subplot(1,2,1)
ax1.set_title('LotArea',fontsize=20)
ax1.set_xlabel('LotArea',fontsize=20)
ax1.set_ylabel('Frequency',fontsize=20)
housingData[['LotArea']].hist(bins=bin_edges1,ax = ax1, xlabelsize=10, ylabelsize=10)

### Removing Outlier

In order to avoid outliers, LotArea greater than 50,000 sq.ft. were eliminated 

In [ ]:
# Dropping lotArea greater than 50000 to remove outlier 
housingData = housingData[housingData.LotArea <= 50000].copy()
housingData.shape

### Adjust for inflation using CPI

In [ ]:
housingData['ADJUSTED_SalesPrice'] = housingData.apply(lambda x: cpi.inflate(x.SalePrice, x.YrSold), axis=1)

##### Prepare the data by separating X and y
##### Dropping unimportant features, such as <>
##### Note that interesting features might be engieered from the dropped features above

In [ ]:
X=housingData[['LotArea','BldgType','HouseStyle','YearBuilt','FullBath','HalfBath','BedroomAbvGr']].copy()
Y=housingData[['ADJUSTED_SalesPrice']]
X.info()

In [ ]:
X.hist(figsize=(15,15))

In [ ]:
Y.hist(figsize=(15,5))

In [ ]:
sns.pairplot(X[['LotArea', 'YearBuilt', 'FullBath', 'HalfBath',
       'BedroomAbvGr']], height=2)

### Split the data into train and test

In [ ]:
# Split the data into a training set and a test set. 
# Any number for the random_state is fine, see 42: https://en.wikipedia.org/wiki/42_(number) 
# We choose to use 20% (test_size=0.2) of the data set as the test set.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
num_features =['LotArea', 'YearBuilt', 'FullBath', 'HalfBath','BedroomAbvGr']
cat_features = ['BldgType','HouseStyle']

## Data pre-processing
We will build a pipeline to do some of the following tasks:

- Missing data
- Feature scaling (important for certain model such as Gradient Descent based models)
- Categorical feature encoding
- Outlier removal
- Transformation
- Custom processing

In [ ]:
# any missing values?
X_train.isnull().sum()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures

# Create the preprocessing pipeline for numerical features
# Pipeline(steps=[(name1, transform1), (name2, transform2), ...]) 
# NOTE the step names can be arbitrary

# Step 1 is feature scaling via standardization - making features look like normal-distributed 
# see sandardization: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)
num_pipeline = Pipeline(
    steps=[
        #('poly',PolynomialFeatures(degree =2)),  # we will tune differet strategies later
        ('scaler', StandardScaler())
        ]
)

# Create the preprocessing pipelines for the categorical features
# There are two steps in this pipeline:
# Step 1: one hot encoding

cat_pipeline = Pipeline(
    steps=[
                ('onehot', OneHotEncoder())
    ]
)

# Assign features to the pipelines and Combine two pipelines to form the preprocessor
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('num_pipeline', num_pipeline, num_features),
        ('cat_pipeline', cat_pipeline, cat_features),
    ]
)

## Model traning, tuning, evaluation and selection

Next, we attach three different models (Linear, Ridge, XGBoost) to the same pre-processing pipeline and tune the some parameters using GridSearch with cross validation. Then, we compare their performance and choose the best model to proceed. 

### Using Linear Regression

In [ ]:
# we show how to use GridSearch with K-fold cross validation (K=10) to fine tune the model
# we use the accuracy as the scoring metric with training score return_train_score=True
from sklearn.model_selection import GridSearchCV

# try Linear Regression
from sklearn.linear_model import LinearRegression

# rf pipeline
pipeline_lr = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LinearRegression()),
])

parameters_lr=[
    {
        'classifier__fit_intercept': [True,False],
        'classifier__copy_X': [True, False],
        'classifier__normalize': [True, False]
    }
]                 

grid_search_lr = GridSearchCV(pipeline_lr,parameters_lr, cv=2)

In [ ]:
grid_search_lr.fit(X_train, y_train)

In [ ]:
# check the best performing parameter combination
grid_search_lr.best_params_

In [ ]:
# build-in CV results keys
sorted(grid_search_lr.cv_results_.keys())

In [ ]:
# best linear regression model test score
grid_search_lr.best_score_

### Using Ridge Classifier

In [ ]:
from sklearn.linear_model import Ridge

# rf pipeline
pipeline_rg = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('clf_RG', Ridge()),
])

parameters_rg=[
    {
        'clf_RG__alpha': [0,0.2,0.01,1.0],
        'clf_RG__copy_X': [True, False],
        'clf_RG__fit_intercept': [True, False]
    }
]                 

grid_search_rg = GridSearchCV(pipeline_rg,parameters_rg, cv=5)

In [ ]:
grid_search_rg.fit(X_train, y_train)

In [ ]:
# best linear regression model test score
grid_search_rg.best_score_

In [ ]:
# best test score
print('best linear regression score is: ', grid_search_lr.best_score_)
print('best Ridge classifier score is: ', grid_search_rg.best_score_)

In [ ]:
# select the best model
# the best parameters are shown, note SimpleImputer() implies that mean strategry is used
best_model = grid_search_lr.best_estimator_

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
# final test on the testing set
# To predict on new data: simply calling the predict method 
# the full pipeline steps will be applied to the testing set followed by the prediction
y_pred = best_model.predict(X_test)

# calculate accuracy, Note: y_test is the ground truth for the tesing set
# we have similiar score for the testing set as the cross validation score - good

#print('Accuracy Score :' (accuracy_score(y_test, y_pred)))

In [ ]:
#===========   R-square and other metrics ===================
r_square= metrics.r2_score(y_test, y_pred)
mae_y = metrics.mean_absolute_error(y_test, y_pred)
mse_y = metrics.mean_squared_error(y_test, y_pred)
rmse_y = np.sqrt(metrics.mean_squared_error(y_test, y_pred))

print("Linear::r_square={0}::mean_absolute_error={1}::mean_square_error={2}::sqrt_mean_square_error={3}::".format(r_square,mae_y,mse_y,rmse_y))

## Feature Importance

Given that we are using pipeline and one-hot encoding, the feature importance scores are not very straightforward to get. The following code shows how to get the feature importance scores from the Linear regression and create a plot.

In [ ]:
best_model.named_steps

In [ ]:
best_model.named_steps['preprocessor']

In [ ]:
i = best_model.named_steps['classifier'].coef_
i

In [ ]:
best_model['preprocessor'].transformers_

In [ ]:
# get columnTransformer
best_model[0] 

In [ ]:
best_model[0].transformers_

In [ ]:
num_original_feature_names = best_model[0].transformers_[0][2]
num_original_feature_names

In [ ]:
cat_original_feature_names = best_model[0].transformers_[1][2]
cat_original_feature_names

In [ ]:
cat_new_feature_names = list(best_model[0].transformers_[1][1]['onehot'].get_feature_names(cat_original_feature_names))
cat_new_feature_names

In [ ]:
feature_names = num_original_feature_names + cat_new_feature_names
feature_names

In [ ]:
r = pd.DataFrame(i, index=feature_names, columns=['importance'])
r

In [ ]:
r.sort_values('importance', ascending=False)

In [ ]:
r.sort_values('importance', ascending=False).plot.bar()

## Remove unimportant Features

## Persists the model

In [ ]:
# Save the model as a pickle file
import joblib
joblib.dump(best_model, "Housing.pickle")

In [ ]:
# Load the model from a pickle file
saved_linear_clf = joblib.load("Housing.pickle")
saved_linear_clf

In [ ]:
housePrice = pd.DataFrame(
    {      
        'BldgType': ['1Fam'], 
        'HouseStyle': ['1Story'],
        'BedroomAbvGr': [4],
        'HalfBath': [1],
        'FullBath': [1],
        'LotArea': [10000],
        'YearBuilt': [1980],
    }
)
housePrice

In [ ]:
#Load Test Data
testhousingData=pd.read_csv('Data/test.csv')
testhousingData.head()

testhousingData.head()
testhousingData_df=testhousingData[['LotArea','BldgType','HouseStyle','YearBuilt','FullBath','HalfBath','BedroomAbvGr']].copy() #,'

In [ ]:
pred1 = saved_linear_clf.predict(testhousingData_df)

In [ ]:
pred1

## Test Datasets

In [ ]:
newark_df=pd.read_csv('Data/Delaware - Newark.csv')
newark_df.head()

In [ ]:
bear_df=pd.read_csv('Data/Delaware - Bear.csv')
bear_df.head()

In [ ]:
wilmigton_df=pd.read_csv('Data/Delaware - Wilmington.csv')
wilmigton_df.head()

In [ ]:
ames_df=pd.read_csv('Data/IA - Ames.csv')
ames_df.head()